## Imports

In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob

## Read-in

In [2]:
# Get today's date
today = datetime.now()

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Aug. 5th Aug. 12th


In [3]:
filename = glob.glob('redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [8]:
# # Filter dates between. If only using data looking back one week, this shouldn't change the df
# df = df[(df['SOLD DATE'] >= '2024-06-10') & (df['SOLD DATE'] <= '2024-06-17')]

In [7]:
df.sort_values(ascending=False, by='SOLD DATE')

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
48,PAST SALE,2024-08-09,Condo/Co-op,104 SW 9th St #1001,Miami,FL,33130.0,435000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/104-SW-9th-St-...,MARMLS,A11592108,N,Y,25.764988,-80.195921
13,PAST SALE,2024-08-09,Condo/Co-op,8775 Park Blvd #104,Miami,FL,33172.0,280000.0,1.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/8775-Park-Blvd...,MARMLS,A11600794,N,Y,25.773541,-80.338252
28,PAST SALE,2024-08-09,Condo/Co-op,8045 NW 7th St #2210,Miami,FL,33126.0,295000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/8045-NW-7th-St...,MARMLS,A11611797,N,Y,25.777233,-80.325223
30,PAST SALE,2024-08-09,Condo/Co-op,761 Euclid Ave #1,Miami Beach,FL,33139.0,201000.0,0.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/761-Eucl...,MARMLS,A11608572,N,Y,25.778233,-80.134716
31,PAST SALE,2024-08-09,Condo/Co-op,761 Euclid Ave #6,Miami Beach,FL,33139.0,193000.0,0.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/761-Eucl...,MARMLS,A11608537,N,Y,25.778233,-80.134716
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,PAST SALE,2024-08-05,Condo/Co-op,1050 Brickell Ave #1808,Miami,FL,33131.0,422000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1050-Brickell-...,MARMLS,A11564433,N,Y,25.763305,-80.191747
57,PAST SALE,2024-08-05,Condo/Co-op,10000 NW 80th Ct #2415,Hialeah Gardens,FL,33016.0,239000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Hialeah-Gardens/1000...,MARMLS,A11565680,N,Y,25.864811,-80.329927
53,PAST SALE,2024-08-05,Condo/Co-op,13529 SW 62nd St #115,Miami,FL,33183.0,485000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/13529-SW-62nd-...,MARMLS,A11557838,N,Y,25.710146,-80.413264
47,PAST SALE,2024-08-05,Condo/Co-op,2831 S Bayshore Dr #804,Miami,FL,33133.0,1390000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2831-S-Bayshor...,MARMLS,A11590764,N,Y,25.729356,-80.238554


In [8]:
df = df.sort_values(by='PRICE',ascending=False)
df_top_ten = df.head(10)

## Current Week's Values

In [9]:
# Number of condos sold
current_week_condo_sales = len(df)
print(len(df))

92


In [10]:
# Average sale price
average_sale_price = round(df['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$678,200


In [11]:
# Average PSF
average_price_psf = round(df['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$530


In [12]:
# Total sales dollar volume
current_week_condo_sales_total = df['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$62,394,400


In [14]:
# # Define the date range
# start_date = '2024-05-20'
# end_date = '2024-05-26'

# # Filter rows where 'SOLD DATE' is within the specified range
# second_week_df = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

## Previous Week's Values

In [13]:
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

Input Previous Week Condo Sales Total (number units sold):
131
Input Previous Week Condo Average Sales Price:
673346
Input Previous Week Condo Average PSF:
510
Input Previous Week Condo Sales total (ex: 198_000_000)
105_000_000


## Clean Data

In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [14]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [15]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [16]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'RANK', 'COLOR'],
      dtype='object')

In [17]:
pd.set_option('display.max_columns',None)

In [18]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
23,PAST SALE,2024-08-08,Condo/Co-op,18501 Collins Ave #2201,Sunny Isles Beach,FL,33160.0,7200000.0,4.0,5.5,TURNBERRY OCEAN CLUB,3252.0,NaN,2020.0,NaN,2214.0,6063.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11601770,N,Y,25.94772,-80.120228,1,orange


In [19]:
def popup_html(row):
    building_name = row['LOCATION']
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(building_name) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [20]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} - {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [21]:
m.save('index.html')

## Data snagger

In [22]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [23]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR
23,PAST SALE,2024-08-08,Condo/Co-op,18501 Collins Ave #2201,Sunny Isles Beach,FL,33160.0,7200000.0,4.0,5.5,TURNBERRY OCEAN CLUB,3252.0,NaN,2020.0,NaN,2214.0,6063.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11601770,N,Y,25.94772,-80.120228,1,orange


In [ ]:
# , {df_top_ten[df_top_ten['PRICE'].idxmax()]['ADDRESS']} | Price ${df_top_ten[df_top_ten['PRICE'].idxmax()]['PRICE']:,.0f}

In [24]:
df_top_ten['PRICE'].iloc[0]

7200000.0

In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [25]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/condo_sales_week_ending_08122024


# CREATE TEMPLATE 

In [26]:
muni_set = set(df_top_ten['CITY'])

In [27]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [28]:
df_top_ten.reset_index(inplace=True,drop=True)

In [29]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [30]:
top_sale

'TURNBERRY OCEAN CLUB at 18501 Collins Ave #2201 in Sunny Isles Beach'

In [31]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [32]:
# df_top_ten

## RESUME HERE

In [33]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops Miami-Dade weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f}M to ${df_top_ten.at[0,'PRICE']:,.0f}M
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m Miami-Dade County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m Miami-Dade County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Adam Farence
\033[1mRESEARCH:\033[0m 
\033[1mSocial:\033[0m #MiamiDade condo sales XXXXX
\033[1mART:\033[0m

Interactive map

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, Miami-Dade County, weekly condo sales, {', '.join(str(x) for x in muni_set)}

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

Miami-Dade County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} sales average from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: TURNBERRY OCEAN CLUB closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,250,000M to $7,200,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Miami, Miami Beach, Sunny Isles Beach, Aventura, Coconut Grove

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 92 condo sales totaling $62,394,400 million from Aug. 5th to Aug. 12th. The previous week, brokers closed 131 condo sales totaling $105,000,000.

Last week’s units sold for an average of $678,200, higher than the $673,346 sales average from the previous week. The a

In [34]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [35]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [36]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf")

print(story_checklist)


HED: TURNBERRY OCEAN CLUB closing tops Miami-Dade weekly condo sales 
DEK: Top 10 sales ranged from $1,250,000M to $7,200,000M
FEATURED HED:
SEO HED: Miami-Dade County Weekly Condo Report 
SEO DESCRIPTION: Miami-Dade County’s condo sales XXXXXXX
AUTHOR: Adam Farence
RESEARCH: 
Social: #MiamiDade condo sales XXXXX
ART:

Interactive map

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Miami-Dade County, weekly condo sales, Miami, Miami Beach, Sunny Isles Beach, Aventura, Coconut Grove

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Miami-Dade County’s NEWS PEG HERE.

Brokers closed 92 condo sales totaling $62,394,400 million from Aug. 5th to Aug. 12th. The previous week, brokers closed 131 condo sales totaling $105,000,000.

Last week’s units sold for an average of $678,200, higher than the $673,346 sales average from the previous week. The a

In [44]:
print(df_top_ten['URL'].iloc[0])

https://www.redfin.com/FL/Sunny-Isles-Beach/18501-Collins-Ave-33160/unit-2201/home/174682499


In [45]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Miami-Beach/4779-Collins-Ave-33140/unit-2201/home/43250708


In [46]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Miami/3064-Ohio-St-33133/unit-3064/home/43474870


In [47]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Miami/325-S-Biscayne-Blvd-33131/unit-2323/home/43331516


In [48]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Sunny-Isles-Beach/18501-Collins-Ave-33160/unit-2201/home/174682499


In [54]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Aventura/3201-NE-183rd-St-33160/unit-1805/home/43438736


In [50]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,RANK,COLOR,FULL_NAME
0,PAST SALE,2024-08-08,Condo/Co-op,18501 Collins Ave #2201,Sunny Isles Beach,FL,33160.0,7200000.0,4.0,5.5,TURNBERRY OCEAN CLUB,3252.0,NaN,2020.0,NaN,2214.0,6063.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/18...,MARMLS,A11601770,N,Y,25.947720,-80.120228,1,orange,TURNBERRY OCEAN CLUB at 18501 Collins Ave #220...
1,PAST SALE,2024-08-07,Condo/Co-op,4779 Collins Ave #2201,Miami Beach,FL,33140.0,2980000.0,3.0,3.0,BLUE DIAMOND CONDO,1980.0,NaN,2000.0,NaN,1505.0,2757.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4779-Col...,MARMLS,A11608148,N,Y,25.823220,-80.121531,2,blue,BLUE DIAMOND CONDO at 4779 Collins Ave #2201 i...
2,PAST SALE,2024-08-07,Condo/Co-op,2831 S Bayshore Dr #1806,Coconut Grove,FL,33133.0,2200000.0,3.0,2.0,CLUB RESIDENCES AT PARK G,1369.0,NaN,2018.0,NaN,1607.0,3518.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2831-S-Bayshor...,MARMLS,A11633057,N,Y,25.729356,-80.238554,3,blue,CLUB RESIDENCES AT PARK G at 2831 S Bayshore D...
3,PAST SALE,2024-08-05,Condo/Co-op,4779 Collins Ave #3504,Miami Beach,FL,33140.0,1850000.0,2.0,2.0,BLUE DIAMOND CONDO,1550.0,NaN,2000.0,NaN,1194.0,4564.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami-Beach/4779-Col...,MARMLS,A11537008,N,Y,25.823220,-80.121531,4,blue,BLUE DIAMOND CONDO at 4779 Collins Ave #3504 i...
4,PAST SALE,2024-08-06,Condo/Co-op,16901 Collins Ave #2204,Sunny Isles Beach,FL,33160.0,1840000.0,1.0,2.0,JADE SIGNATURE CONDO,1125.0,NaN,2018.0,NaN,1636.0,2125.0,Sold,NaN,NaN,https://www.redfin.com/FL/Sunny-Isles-Beach/16...,MARMLS,A11578905,N,Y,25.932764,-80.121036,5,blue,JADE SIGNATURE CONDO at 16901 Collins Ave #220...
5,PAST SALE,2024-08-08,Condo/Co-op,901 Brickell Key Blvd #307,Miami,FL,33131.0,1725000.0,2.0,2.5,CARBONELL CONDO,1784.0,NaN,2005.0,NaN,967.0,1874.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/901-Brickell-K...,MARMLS,A11470616,N,Y,25.768991,-80.186892,6,blue,CARBONELL CONDO at 901 Brickell Key Blvd #307 ...
6,PAST SALE,2024-08-05,Condo/Co-op,2831 S Bayshore Dr #804,Miami,FL,33133.0,1390000.0,2.0,2.0,CLUB RESIDENCES AT PARK G,978.0,NaN,2018.0,NaN,1421.0,2600.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/2831-S-Bayshor...,MARMLS,A11590764,N,Y,25.729356,-80.238554,7,blue,CLUB RESIDENCES AT PARK G at 2831 S Bayshore D...
7,PAST SALE,2024-08-07,Condo/Co-op,3201 NE 183rd St #1805,Aventura,FL,33160.0,1375000.0,2.0,3.5,THE PENINSULA CONDO,2970.0,NaN,2003.0,NaN,463.0,2695.0,Sold,NaN,NaN,https://www.redfin.com/FL/Aventura/3201-NE-183...,MARMLS,A11496448,N,Y,25.945719,-80.135657,8,blue,THE PENINSULA CONDO at 3201 NE 183rd St #1805 ...
8,PAST SALE,2024-08-05,Condo/Co-op,325 S Biscayne Blvd #2323,Miami,FL,33131.0,1283000.0,3.0,2.0,ONE MIAMI WEST CONDO,1792.0,NaN,2005.0,NaN,716.0,2042.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/325-S-Biscayne...,MARMLS,A11568908,N,Y,25.771705,-80.186373,9,blue,ONE MIAMI WEST CONDO at 325 S Biscayne Blvd #2...
9,PAST SALE,2024-08-05,Condo/Co-op,3064 Ohio St #3064,Miami,FL,33133.0,1250000.0,3.0,2.5,3060 OHIO STREET CONDO,2374.0,NaN,2007.0,NaN,527.0,300.0,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/3064-Ohio-St-3...,MARMLS,A11535129,N,Y,25.734069,-80.247901,10,blue,3060 OHIO STREET CONDO at 3064 Ohio St #3064 i...


## Time on Market Calculator

In [52]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 6, 17) ## List (Earlier) date
date2 = datetime(2024, 8, 7) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

51
